In [1]:
import os
import sys
import re
import time
from github import Github
from random import randint
from pprint import pprint

# Credentials

In [2]:
from pymongo import MongoClient
pw_file = 'credentials/pw.txt'
if os.path.exists(pw_file): 
    with open(pw_file, 'r') as f:
        email, indeed_pw = f.readline().strip().split(', ')
        username, pia_pw = f.readline().strip().split(', ')
        pub_ip, mongo_usr, mongo_usr_pw = f.readline().strip().split(', ')

# Connect to MongoDB

In [3]:
# connect to ec2 mongo client
client = MongoClient('{0}:27017'.format(pub_ip))

# get reference to  resume_db
db = client['github_db']

# authenticate user for database
db.authenticate(mongo_usr, mongo_usr_pw)

True

# Create a MongoDB collection

In [4]:
#db.drop_collection('github_repos_docs')

In [5]:
print('Collections in database: {0}'.format(db.collection_names()))
for name in db.collection_names():
    print('Number of documents in \"{0}\" collection: {1}'.format(name, db[name].count()))

Collections in database: ['github_users', 'github_repos_meta', 'github_repos_docs']
Number of documents in "github_users" collection: 0
Number of documents in "github_repos_meta" collection: 0
Number of documents in "github_repos_docs" collection: 0


In [6]:
# create a collection in the 'github_db' database
lst_collections = ['github_users', 'github_repos_meta', 'github_repos_docs']

for collection in lst_collections:
    if collection not in db.collection_names():
        db.create_collection(collection)

In [7]:
print('Collections in database: {0}'.format(db.collection_names()))
for name in db.collection_names():
    print('Number of documents in \"{0}\" collection: {1}'.format(name, db[name].count()))

Collections in database: ['github_users', 'github_repos_meta', 'github_repos_docs']
Number of documents in "github_users" collection: 0
Number of documents in "github_repos_meta" collection: 0
Number of documents in "github_repos_docs" collection: 0


# Get authenticated access to Github for higher requests limit

In [8]:
# import github private token
with open('credentials/token.txt', 'r') as infile:
    token = infile.readline().strip()
    user = infile.readline().strip()

git_client = Github(token)

# Rate limits

In [9]:
def get_remaining(_type ='core'):
    '''
    _type = 'core' or 'search'
    Return: tuple of remaining rate limit quantity, and time till reset
    '''
    
    rate_limit = git_client.get_rate_limit()
    raw = dict(rate_limit.raw_data)
    
    remaining = int(raw['resources'][_type]['remaining'])
    reset = int(raw['resources'][_type]['reset'])
    
    return (remaining, reset)

In [10]:
print('Resource rate limit remaining: {0}'.format(get_remaining('core')))
print('Search rate limit remaining: {0}'.format(get_remaining('search')))

Resource rate limit remaining: (4674, 1472823467)
Search rate limit remaining: (30, 1472821561)


# REMOVE ME - ONLY FOR DEV

In [20]:
USER = 'ptwobrussell'
REPO = 'Mining-the-Social-Web'

user = git_client.get_user(USER)
repo = user.get_repo(REPO)

# Repo Metadata Functions

In [54]:
def repo_subscribers(repo):
    '''
    IN: github repo object
    RETURN: dictionary of subscribers keyed by repo full_name
    '''
    repo_dict = {}
    try:
        repo_dict[repo.full_name] = [f.login for f in repo.get_watchers()]
    except:
        repo_dict[repo.full_name] = ''
        
    return repo_dict

In [55]:
b = list(repo_subscribers(repo).values())
len(b[0])

1074

In [ ]:
repo.get_watchers

In [27]:
def repo_stargazers(repo):
    '''
    IN: github repo object
    RETURN: dictionary of stargazers keyed by repo full_name
    '''
    repo_dict = {}
    try:
        repo_dict[repo.full_name] = [f.login for f in repo.get_stargazers()]
    except:
        repo_dict[repo.full_name] = ''

    return repo_dict

In [53]:
c = list(repo_stargazers(repo).values())
len(c[0])

1074

In [13]:
def repo_contributors(repo):
    '''
    IN: github repo object
    RETURN: dictionary of contributors keyed by repo full_name
    '''
    repo_dict = {}
    try:
        repo_dict[repo.full_name] = [f.login for f in repo.get_contributors()]
    except:
        repo_dict[repo.full_name] = ''
        
    return repo_dict

In [14]:
def repo_metadata(repo):
    repo_keys = ['id', 'owner', 'name', 'full_name', 'description', 'fork', 'html_url', 'homepage', 
             'language', 'forks_count', 'size', 'open_issues_count', 'has_issues', 'has_wiki', 'has_downloads', 
             'pushed_at', 'created_at', 'subscribers_count', 'stargazers_count']
    
    owner_keys = ['type', 'login', 'id', 'site_admin']
    
    # get returned payload raw data into dictionary
    raw_dict = dict(repo.raw_data)
    
    # trim repo by keys listed above
    repo_dict = {k: raw_dict[k] for k in repo_keys}
    
    # trim owner by keys listed above
    owner_dict = {k: raw_dict['owner'][k] for k in owner_keys}
    repo_dict['owner'] = owner_dict
    
    return repo_dict

In [15]:
#a = repo_metadata(repo)

# User Metadata Functions

In [16]:
def user_following(user):
    '''
    IN: github user object
    RETURN: dictionary of users (login name) followed by user (passed) 
            keyed by passed user's login
    '''
    user_dict = {}
    try:
        user_dict[user.login] = [f.name for f in user.get_following()]
    except:
        user_dict[user.login] = ''
        
    return user_dict

In [17]:
def user_followers(user):
    '''
    IN: github user object
    RETURN: dictionary of users (login name) who follow user (passed)
            keyed by passed user's login
    '''
    user_dict = []
    try:
        user_dict[user.login] = [f.login for f in user.get_followers()]
    except:
        user_dict[user.login] = ''
    
    return user_dict

In [18]:
def user_starred(user):
    '''
    IN: github user object
    RETURN: dictionary of repos (full_name) starred by user (passed)
            keyed by passed user's login
    '''
    user_dict = {}
    try:
        user_dict[user.login] = [f.full_name for f in user.get_starred()]
    except:
        user_dict[user.login] = ''
        
    return user_dict

# User data

In [19]:
def user_metadata(user):
    user_keys = ['email', 'followers', 'hireable', 'login', 'id', 'bio', 'avatar_url', 'company', 
                 'updated_at', 'type', 'created_at', 'name', 'location', 'html_url', 'public_repos', 
                 'blog', 'public_gists', 'following']

    # get returned payload raw data into dictionary
    raw_dict = dict(user.raw_data)
    
    # trim user by keys listed above
    user_dict = {k: raw_dict[k] for k in user_keys}
    
    return user_dict

# Upsert MongoDB Document by Repo Full Name

In [ ]:
def repo_doc_upsert(repo_fullname, file_name, raw_file_contents, file_extension, doc_num):
    '''
    '''
    # strip newline and tab chars from raw file content
    stripped_file_contents = raw_file_contents.replace('\n', ' ')
    stripped_file_contents = stripped_file_contents.replace('\t', ' ')
    
    repo_chk = db['github_repos'].find({repo_fullname: {"$exists": True}})
    
    if not repo_chk:
        db['github_repos'].insert_one({'_id':repo_fullname})
        
    try:
        db['github_repos'].update_one({'_id' : repo_fullname}, 
        {
            '$set':{
                doc_num: {
                        'file_name': file_name,
                        #'raw_file': raw_file_contents,
                        'stripped_file': stripped_file_contents,
                        'extension': file_extension,
                        }
            }
        }, upsert=True
        )
    except:
        pass
    return None

# Upsert MongoDB Repo Metadata by Repo Full Name

In [ ]:
def repo_meta_upsert(repo_fullname, repo_metadata):
    repo_chk = db['github_repos'].find({repo_fullname: {"$exists": True}})
    
    if not repo_chk:
        db['github_repos'].insert_one({'_id':repo_fullname})
        
    try:
        db['github_repos'].update_one({'_id' : repo_fullname}, 
        {
            '$set':{
                db['github_repos'].insert_one(repo_metadata)
            }
        }, upsert=True
        )
    except:
        pass
    return None

In [ ]:
def upload_files(repo, doc_ct, _path='.'):
    '''
    IN: github repo object and path to folder
    RETRUN: all files from repo except those in the exclusion list
    '''
    # only files with these extensions
    extensions = ['py']

    # grab all contents in the main directory
    dir_contents = repo.get_dir_contents(_path)
    repo_fullname = repo.full_name

    for content in dir_contents:
        # if item is a directory then recursively navigate lower to get files inside
        if content.type == 'dir':
            upload_files(repo, doc_ct, _path=content.path)

        else:
            doc_ct += 1
            
            # get file extension
            file_split = content.name.split('.')
            file_ext = file_split[-1]

            if file_ext in extensions:
                # try to decode, but return blank if fail
                try:
                    raw_file_content = content.decoded_content.decode(errors='replace') # decode githubs compression
                except:
                    raw_file_content = ''
                    
                # file and mongod doc names
                file_name = content.name
                doc_name = 'doc_{0}'.format(doc_ct)
                
                # if file is not too large (>16mb), add to collection
                file_size = sys.getsizeof(raw_file_content)
                if file_size < 16000000:
                    repo_doc_upsert(repo_fullname, file_name, raw_file_content, file_ext, doc_name)
                else:
                    print('FILE SIZE LIMIT: {0} -- {1}'.format(file_name, file_size))
    return None

# Search Github for 'Python' repositories

In [ ]:
import time

for repo in git_client.search_repositories('python')[5:6]:
    doc_ct = 0
    
    # ensure its a python tagged dirctory
    if repo.language.lower() == 'python':
        
        #############################
        # INSERT FURTHER CALLS HERE #
        
        rate, _ = get_remaining('core')
        
        if (rate > 10):
            print('Working on {0}'.format(repo.full_name))
            upload_files(repo, doc_ct)
            
            # insert metadata
            repo_metadata = repo_data(repo)
            repo_meta_upsert(repo.full_name, repo_metadata)
            
        else:
            print('Waiting to continue...')
            time.sleep(1800)
            
            if (rate > 10):
                print('Retrying {0}'.format(repo.full_name))
                upload_files(repo, doc_ct)
            else:
                print('Waiting to continue...')
                time.sleep(1800)
            
                if (rate > 10):
                    print('Final retry {0}'.format(repo.full_name))
                    upload_files(repo, doc_ct)
                else:
                    print('Waiting to continue...')
                    time.sleep(900)
            
        #############################
        
    # don't get cut off
    time.sleep(randint(2,9))
    
print('DONE')

In [ ]:
for cursor in db['github_repos'].find():
    pprint(cursor)

In [ ]:
_ = db.drop_collection('github_repos')
_ = db.create_collection('github_repos')

In [ ]:
_ = db['github_repos'].insert_one({'_id':'joe011/python'})